In [2]:
import json
import re
import ssl

import certifi
import geopy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from geopy import Nominatim
from datetime import datetime, date
import ipykernel

In [3]:
# to potrzebne, bo inaczej wyskakuje GeocoderServiceError, ale ogólnie bardzo niewskazane XD
ssl._create_default_https_context = ssl._create_unverified_context

In [45]:
df = pd.read_csv("data/data_updated.csv", na_values="--")
df["date"] = df["data"] + ".23" + " " + df["czas_rozkladowy"]
#any(df.duplicated(subset = ["nazwa_pociagu", "stacja_poczatkowa", "stacja_koncowa", "stacja_pomiaru", "data"]))
df.head(30)
df["date"] = pd.to_datetime(df["date"],format= "%d.%m.%y %H:%M", errors= "coerce")
df = df.loc[(df["date"] >= "13.03.23")&(df["date"] < "27.03.23")]
df.to_csv("test.csv")
df["date"]


92476    2023-03-21 08:42:00
92477    2023-03-21 08:47:00
92478    2023-03-21 09:02:00
92479    2023-03-21 09:16:00
92480    2023-03-21 09:27:00
                 ...        
194150   2023-03-14 23:19:00
194151   2023-03-14 23:32:00
194152   2023-03-14 23:56:00
194153   2023-03-14 00:06:00
194154   2023-03-14 00:23:00
Name: date, Length: 82114, dtype: datetime64[ns]

In [46]:
df.shape

(82114, 9)

In [49]:
stations  = df["stacja_pomiaru"].unique().tolist()
pd.DataFrame(stations).to_csv("stations.csv", index = False)
#stations

### Create graph nodes

In [8]:
pos = {}
len_stations = len(stations)
loc = Nominatim(user_agent="GetLoc")
for index,station in enumerate(stations):
    print(f"{index+1}/{len_stations}")
    getLoc = loc.geocode(station)
    pos[station] = [getLoc.longitude,getLoc.latitude]
print(pos)
json.dump(pos,open('station_pos.json','w'))

1/465
2/465
3/465
4/465
5/465
6/465
7/465
8/465
9/465
10/465
11/465


KeyboardInterrupt: 

### Create nodes from Martyna's data

In [172]:
df_position = pd.read_csv("data/pociagi i miejscowosci.csv")
stations = df["stacja_pomiaru"].unique()
df_position.head()
missing = []
pos_dict = {}
for station in stations:
    print(station)
    pos_new = df_position.loc[df_position["stacja_pomiaru"] == station ][[ "dlugosc", "szerokosc"]].drop_duplicates(["szerokosc", "dlugosc"]).values.tolist()
    print(pos_new)
    try:
        pos_dict[station] = pos_new[0]
    except:
        missing.append(station)

Kraków Główny
[]
Warszawa Gdańska
[]
Gdynia Główna
[]
Sopot
[[18.5608, 54.4453]]
Gdańsk Oliwa
[]
Gdańsk Wrzeszcz
[]
Gdańsk Główny
[]
Tczew
[[18.7994, 54.0881]]
Malbork
[[19.0286, 54.0358]]
Iława Główna
[]
Warszawa Wschodnia
[]
Warszawa Centralna
[]
Warszawa Zachodnia
[]
Miechów
[[20.0286, 50.3558]]
Włoszczowa Północ
[]
Opoczno Południe
[]
Łódź Fabryczna
[]
Łódź Widzew
[]
Łódź Chojny
[]
Łódź Retkinia
[]
Pabianice
[[19.3592, 51.665]]
Łask
[[19.1336, 51.5914]]
Zduńska Wola
[]
Sieradz
[[18.7322, 51.5961]]
Kalisz
[[18.0869, 51.7611]]
Pleszew
[[17.785800000000002, 51.8964]]
Jarocin
[[17.5017, 51.9725]]
Środa Wielkopolska
[]
Poznań Główny
[]
Elbląg
[[19.4097, 54.1614]]
Pasłęk
[[19.6586, 54.0653]]
Morąg
[[19.9292, 53.9161]]
Olsztyn Zachodni
[]
Olsztyn Główny
[]
Barczewo
[[20.6903, 53.8283]]
Korsze
[[21.1406, 54.1722]]
Kętrzyn
[[21.3747, 54.0772]]
Giżycko
[[21.7717, 54.04]]
Ustka
[[16.8569, 54.5847]]
Słupsk
[[17.0372, 54.4656]]
Sławno
[[16.6775, 54.3581]]
Koszalin
[[16.1819, 54.1903]]
Białogard

In [173]:
loc = Nominatim(user_agent="GetLoc")
for m in missing:
    getLoc = loc.geocode(m)
    pos_dict[m] = [getLoc.longitude,getLoc.latitude]


In [174]:
pos_dict
json.dump(pos_dict,open('station_pos_new.json','w'))

### Create graph connections

In [41]:
temp = df[['stacja_pomiaru','opoznienie']].loc[(df['nazwa_pociagu']=='EIJ 5306/7')&(df['data']=='13.03')]
temp

,stacja_pomiaru,opoznienie
93071,Gdynia Główna,0.0
93072,Sopot,0.0
93073,Gdańsk Oliwa,0.0
93074,Gdańsk Wrzeszcz,1.0
93075,Gdańsk Główny,1.0
93076,Tczew,2.0
93077,Malbork,4.0
93078,Iława Główna,6.0
93079,Warszawa Wschodnia,17.0
93080,Warszawa Centralna,15.0


In [42]:
stations = temp['stacja_pomiaru'].tolist()
delays = temp['opoznienie'].tolist()
start = [[station,delay] for station,delay in zip(stations[:-1],delays[:-1])]
end = [[station,delay] for station,delay in zip(stations[1:],delays[1:])]
print(start)
end

[['Gdynia\xa0Główna', 0.0], ['Sopot', 0.0], ['Gdańsk\xa0Oliwa', 0.0], ['Gdańsk\xa0Wrzeszcz', 1.0], ['Gdańsk\xa0Główny', 1.0], ['Tczew', 2.0], ['Malbork', 4.0], ['Iława\xa0Główna', 6.0], ['Warszawa\xa0Wschodnia', 17.0], ['Warszawa\xa0Centralna', 15.0], ['Warszawa\xa0Zachodnia', 7.0]]


[['Sopot', 0.0],
 ['Gdańsk\xa0Oliwa', 0.0],
 ['Gdańsk\xa0Wrzeszcz', 1.0],
 ['Gdańsk\xa0Główny', 1.0],
 ['Tczew', 2.0],
 ['Malbork', 4.0],
 ['Iława\xa0Główna', 6.0],
 ['Warszawa\xa0Wschodnia', 17.0],
 ['Warszawa\xa0Centralna', 15.0],
 ['Warszawa\xa0Zachodnia', 7.0],
 ['Kraków\xa0Główny', 5.0]]

In [24]:
df

,nazwa_pociagu,stacja_poczatkowa,stacja_koncowa,stacja_pomiaru,data,czas_rozkladowy,czas_przyjazdu,opoznienie,date
20,EIJ 3104/5,Kraków Główny,Warszawa Gdańska,Kraków Główny,27.02,18:04,18:23,19.0,27.02.23 18:04
21,EIJ 3104/5,Kraków Główny,Warszawa Gdańska,Warszawa Gdańska,27.02,20:31,20:48,17.0,27.02.23 20:31
22,EIJ 3104/5,Kraków Główny,Warszawa Gdańska,Kraków Główny,26.02,18:04,18:04,0.0,26.02.23 18:04
23,EIJ 3104/5,Kraków Główny,Warszawa Gdańska,Warszawa Gdańska,26.02,20:31,20:30,-1.0,26.02.23 20:31
223,MPE 31102/3 MARSZAŁEK PIŁSUDSKI,Kraków Główny,Warszawa Gdańska,Kraków Główny,27.02,08:11,08:11,0.0,27.02.23 08:11
...,...,...,...,...,...,...,...,...,...
194150,EIJ 1628/9 OLEŃKA,Warszawa Gdańska,Wrocław Główny,Ozimek,14.03,23:19,23:22,3.0,14.03.23 23:19
194151,EIJ 1628/9 OLEŃKA,Warszawa Gdańska,Wrocław Główny,Opole Główne,14.03,23:32,23:35,3.0,14.03.23 23:32
194152,EIJ 1628/9 OLEŃKA,Warszawa Gdańska,Wrocław Główny,Brzeg,14.03,23:56,00:02,6.0,14.03.23 23:56
194153,EIJ 1628/9 OLEŃKA,Warszawa Gdańska,Wrocław Główny,Oława,14.03,00:06,00:12,6.0,14.03.23 00:06


In [25]:
df.loc[df["stacja_pomiaru"]== "Sopot"]

,nazwa_pociagu,stacja_poczatkowa,stacja_koncowa,stacja_pomiaru,data,czas_rozkladowy,czas_przyjazdu,opoznienie,date
2149,MPE 57106/7 LEDNICA,Gdynia Główna,Poznań Główny,Sopot,27.02,17:57,17:57,0.0,27.02.23 17:57
2161,MPE 57106/7 LEDNICA,Gdynia Główna,Poznań Główny,Sopot,23.02,17:57,18:04,7.0,23.02.23 17:57
2173,MPE 57106/7 LEDNICA,Gdynia Główna,Poznań Główny,Sopot,22.02,17:57,17:57,0.0,22.02.23 17:57
2185,MPE 57106/7 LEDNICA,Gdynia Główna,Poznań Główny,Sopot,21.02,17:57,17:57,0.0,21.02.23 17:57
2197,MPE 57106/7 LEDNICA,Gdynia Główna,Poznań Główny,Sopot,20.02,17:57,17:57,0.0,20.02.23 17:57
...,...,...,...,...,...,...,...,...,...
192940,MPE 35102/3 LUBOMIRSKI,Kraków Główny,Gdynia Główna,Sopot,22.03,18:12,18:21,9.0,22.03.23 18:12
192985,MPE 53102/3 LUBOMIRSKI,Gdynia Główna,Kraków Główny,Sopot,24.03,09:50,09:54,4.0,24.03.23 09:50
193006,MPE 53102/3 LUBOMIRSKI,Gdynia Główna,Kraków Główny,Sopot,23.03,09:50,09:50,0.0,23.03.23 09:50
193027,MPE 53102/3 LUBOMIRSKI,Gdynia Główna,Kraków Główny,Sopot,22.03,09:50,09:50,0.0,22.03.23 09:50


In [70]:
pairs = []
unique_id = df['nazwa_pociagu'].unique().tolist()[:2]
unique_dates = df['data'].unique().tolist()
for id_num,id in enumerate(unique_id):
    print("next train")
    for id_date,date in enumerate(unique_dates):
        df_tmp = df.loc[(df["nazwa_pociagu"]== id)&(df["data"] == date), ["stacja_poczatkowa", "stacja_koncowa", "stacja_pomiaru", "czas_rozkladowy", "opoznienie"]]
        start, end = df_tmp["stacja_poczatkowa"].unique()[0], df_tmp["stacja_koncowa"].unique()[0]
        print(start, end)
        #temp = df_tmp.loc[df_tmp["stacja_pomiaru"]== start]["czas_rozkladowy"].unique()[0]
        #print("temp: ", temp)
        start_time = datetime.strptime(df_tmp.loc[df_tmp["stacja_pomiaru"]== start]["czas_rozkladowy"].unique()[0],"%H:%M")
        end_time = datetime.strptime(df_tmp.loc[(df_tmp["stacja_pomiaru"]== end)]["czas_rozkladowy"].unique()[0],"%H:%M")
        print(start_time.strftime("%H:%M") + "\n" + end_time.strftime("%H:%M"))
        is_overnight =  start_time > end_time
        if not is_overnight:
            stations = df_tmp['stacja_pomiaru'].tolist()
            delays = df_tmp['opoznienie'].tolist()
            print(f'{id_num}/{len(unique_id)},{id_date}/{len(unique_dates)}, {id}')
            for i in range(len(stations)-1):
                if stations[i] != stations[i+1]:
                    try:
                        pair = [stations[i],stations[i+1]]
                        delay = int(delays[i+1]) - int(delays[i])
                    except ValueError:
                        print("got nan!")
                        delay = None
                    time = df["czas_rozkladowy"].loc[(df['nazwa_pociagu']==id)&(df['data']==date)&(df["stacja_pomiaru"]==pair[1])].tolist()[0]
                    pairs.append([id, pair[0],pair[1],date, time, delay])

next train
Wrocław Główny Giżycko
08:42
16:56
0/2,0/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,1/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,2/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,3/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,4/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,5/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,6/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,7/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,8/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,9/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,10/13, MPE 65103/2 MAMRY
Wrocław Główny Giżycko
08:42
16:56
0/2,11/13, MPE 65103/2 MAMRY


IndexError: index 0 is out of bounds for axis 0 with size 0

In [79]:
df.loc[df['nazwa_pociagu']=="EIJ 5306/7"]

,nazwa_pociagu,stacja_poczatkowa,stacja_koncowa,stacja_pomiaru,data,czas_rozkladowy,czas_przyjazdu,opoznienie
24,EIJ 5306/7,Gdynia Główna,Kraków Główny,Gdynia Główna,11.03,12:32,12:35,3.0
25,EIJ 5306/7,Gdynia Główna,Kraków Główny,Sopot,11.03,12:39,12:42,3.0
26,EIJ 5306/7,Gdynia Główna,Kraków Główny,Gdańsk Oliwa,11.03,12:44,12:52,8.0
27,EIJ 5306/7,Gdynia Główna,Kraków Główny,Gdańsk Wrzeszcz,11.03,12:49,12:57,8.0
28,EIJ 5306/7,Gdynia Główna,Kraków Główny,Gdańsk Główny,11.03,12:55,13:03,8.0
...,...,...,...,...,...,...,...,...
159547,EIJ 5306/7,Gdynia Główna,Kraków Główny,Iława Główna,22.03,13:55,13:58,3.0
159548,EIJ 5306/7,Gdynia Główna,Kraków Główny,Warszawa Wschodnia,22.03,15:27,15:29,2.0
159549,EIJ 5306/7,Gdynia Główna,Kraków Główny,Warszawa Centralna,22.03,15:36,15:36,0.0
159550,EIJ 5306/7,Gdynia Główna,Kraków Główny,Warszawa Zachodnia,22.03,15:54,15:56,2.0


In [119]:
p_test = pd.DataFrame(pairs, columns=["nazwa_pociagu", "stacja_wyjazdu", "stacja_przyjazdu", "data", "czas_rozkladowy", "opoznienie_netto"]).sort_values(by = ["nazwa_pociagu","data", "czas_rozkladowy"])
p_test.sort_values(by= ["data"])
p_test.head(50)
p_test.isnull().values.any()

True

In [87]:
p = pd.DataFrame(pairs, columns=["nazwa_pociagu", "stacja_wyjazdu", "stacja_przyjazdu", "data", "czas_rozkladowy", "opoznienie_netto"]).sort_values(by = ["nazwa_pociagu","data", "czas_rozkladowy"])
p.sort_values(by= ["data"])
p.head(50)

,nazwa_pociagu,stacja_wyjazdu,stacja_przyjazdu,data,czas_rozkladowy,opoznienie_netto
23976,ECE 14000/1 SILESIA,Warszawa Wschodnia,Warszawa Centralna,01.03,05:07,0.0
23977,ECE 14000/1 SILESIA,Warszawa Centralna,Warszawa Zachodnia,01.03,05:19,0.0
23978,ECE 14000/1 SILESIA,Warszawa Zachodnia,Grodzisk Mazowiecki,01.03,05:37,2.0
23979,ECE 14000/1 SILESIA,Grodzisk Mazowiecki,Opoczno Południe,01.03,06:20,5.0
23980,ECE 14000/1 SILESIA,Opoczno Południe,Włoszczowa Północ,01.03,06:49,-2.0
23981,ECE 14000/1 SILESIA,Włoszczowa Północ,Zawiercie,01.03,07:27,5.0
23982,ECE 14000/1 SILESIA,Zawiercie,Dąbrowa Górnicza,01.03,07:46,-4.0
23983,ECE 14000/1 SILESIA,Dąbrowa Górnicza,Sosnowiec Główny,01.03,07:58,0.0
23984,ECE 14000/1 SILESIA,Sosnowiec Główny,Katowice,01.03,08:09,-3.0
23985,ECE 14000/1 SILESIA,Katowice,Tychy,01.03,08:28,-1.0


In [121]:
p.to_csv("data/connections.csv", encoding="utf-8-sig")
p.head(30)
p_test.to_csv("data/connections_test.csv", encoding="utf-8-sig", na_rep = "null")

In [93]:
print(len(p[['stacja_wyjazdu','stacja_przyjazdu']]))
print(len(p[['stacja_wyjazdu','stacja_przyjazdu']].drop_duplicates()))
unique_connections = p[['stacja_wyjazdu','stacja_przyjazdu']].drop_duplicates()
unique_connections = [(conn[1],conn[2]) for conn in unique_connections.itertuples()]
unique_connections

183622
1345


[('Warszawa\xa0Wschodnia', 'Warszawa\xa0Centralna'),
 ('Warszawa\xa0Centralna', 'Warszawa\xa0Zachodnia'),
 ('Warszawa\xa0Zachodnia', 'Grodzisk\xa0Mazowiecki'),
 ('Grodzisk\xa0Mazowiecki', 'Opoczno\xa0Południe'),
 ('Opoczno\xa0Południe', 'Włoszczowa\xa0Północ'),
 ('Włoszczowa\xa0Północ', 'Zawiercie'),
 ('Zawiercie', 'Dąbrowa\xa0Górnicza'),
 ('Dąbrowa\xa0Górnicza', 'Sosnowiec\xa0Główny'),
 ('Sosnowiec\xa0Główny', 'Katowice'),
 ('Katowice', 'Tychy'),
 ('Tychy', 'Rybnik'),
 ('Rybnik', 'Wodzisław\xa0Śląski'),
 ('Wodzisław\xa0Śląski', 'Chałupki'),
 ('Chałupki', 'Bohumin'),
 ('Warszawa\xa0Zachodnia', 'Zawiercie'),
 ('Zawiercie', 'Sosnowiec\xa0Główny'),
 ('Tychy', 'Pszczyna'),
 ('Pszczyna', 'Zebrzydowice'),
 ('Zebrzydowice', 'Petrovice\xa0u\xa0Karvine'),
 ('Terespol', 'Biała\xa0Podlaska'),
 ('Biała\xa0Podlaska', 'Międzyrzec\xa0Podlaski'),
 ('Międzyrzec\xa0Podlaski', 'Łuków'),
 ('Łuków', 'Siedlce'),
 ('Siedlce', 'Mińsk\xa0Mazowiecki'),
 ('Mińsk\xa0Mazowiecki', 'Warszawa\xa0Gdańska'),
 ('Warszaw

In [97]:
p.loc[(p["stacja_wyjazdu"]== "Szczecinek")]

,nazwa_pociagu,stacja_wyjazdu,stacja_przyjazdu,data,czas_rozkladowy,opoznienie_netto
155662,EIE 3806/7 MALCZEWSKI,Szczecinek,Białogard,01.03,14:50,-2.0
155643,EIE 3806/7 MALCZEWSKI,Szczecinek,Białogard,02.03,14:50,-2.0
155624,EIE 3806/7 MALCZEWSKI,Szczecinek,Białogard,03.03,14:50,-1.0
155700,EIE 3806/7 MALCZEWSKI,Szczecinek,Białogard,04.03,14:50,22.0
155605,EIE 3806/7 MALCZEWSKI,Szczecinek,Białogard,05.03,14:50,-2.0
...,...,...,...,...,...,...
111872,MPE 86103/2 SZKUNER,Szczecinek,Jastrowie,22.03,08:40,1.0
111855,MPE 86103/2 SZKUNER,Szczecinek,Jastrowie,23.03,08:40,2.0
111838,MPE 86103/2 SZKUNER,Szczecinek,Jastrowie,24.03,08:40,0.0
111821,MPE 86103/2 SZKUNER,Szczecinek,Jastrowie,27.03,08:40,0.0
